In [53]:
import keras
from keras.utils import np_utils
import rasterio
import os,sys
import random
import numpy as np
import math
from pathlib import Path
import cv2
from skimage.transform import resize
from itertools import compress



#os.chdir('/home/daniel/R/landuse/Sentinel_models')

epochs = 3
classes = 5 # Remember: 'not labeled' is also a class!
batch_size = 1
model_name = 'model/model_noreflect'
save_each = 3
path = Path('/Users/datalab1/Lars/Panorama_Amsterdam/')


files_labels = os.listdir(os.path.join(path,'labeled' ))
print(files_labels)

# np_utils.to_categorical is faster
# def get_one_hot(targets, nb_classes):
#     print('[get_one_hot]',np.shape(targets),np.shape(nb_classes))
# #     print(nb_classes)
#     res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
#     return res.reshape(list(targets.shape)+[nb_classes])


def generator():
    while(True):
        #get all labels and shuffle them
        files_labels = os.listdir(os.path.join(path,'labeled' ))
        random.shuffle(files_labels)
        
        #construct filenames of raw data       
        files_raws = list()
        for label in files_labels:
            files_raws.append(label.replace('.txt', '.jpg'))  

        #calculate how long it takes to loop through all files            
        n = int(math.floor(len(files_labels)/batch_size))      
        
        #construct number of rotations and reflections per file
#         num_rotations =  np.random.choice([0,1,2,3], size = n , replace = True)
        reflect = np.random.choice([False, True], size = n, replace = True)
                    
        for i in np.arange(n):        
            
            labels = list() # Label is empty list every loop because a single label and image are passed to the model at the end
            for j in np.arange(batch_size):
                label = os.path.join(path,'labeled',files_labels[batch_size*i + j])
                label = path / 'labeled' / files_labels[batch_size*i + j]
#                 print('[label]',files_labels[batch_size*i + j])
                label = np.genfromtxt(label, delimiter = ',')
#                 print(np.mean(label))
                label = np.rint(resize(label, (128, 256))).astype(int)
#                 print(np.mean(label))
#                 print('label before one hot vector',np.shape(label))
                label = np_utils.to_categorical(label, classes)
                label = label.astype(np.int32)
                label = np.expand_dims(label, axis = 0)
                labels.append(label)
#                 print('end of label routine',label.shape)
            
            labels = np.concatenate(labels, axis = 0)            

            ims = list() # ims is empty list every loop because a single label and image are passed to the model at the end
            
            for j in np.arange(batch_size):
                imagePath=os.path.join(path,'target',files_raws[batch_size*i +j])
                imagePath = str(path / "target" / files_raws[batch_size*i +j]) # cv2 wants a string as pathname, without str() this would be a pathlib class
                image = cv2.imread(imagePath)
#                 print('before image resize',np.shape(image))
                image = cv2.resize(image, (256, 128))
#                 image = np.expand_dims(image, axis=0)
                ims.append(image)
            
#           Do a random mirror over x axis 
            if reflect[batch_size*i]:
                print(np.shape(ims))
                ims=np.flip(ims, axis = 1)
                labels=np.flip(labels, axis = 1)
            ims = np.expand_dims(image, axis=0)

#             #do a random rotation !!!!!!!!!!Does not do anything yet!!!!!!!!!!!
#             for k in range(num_rotations[batch_size*i]):
#                 r =  np.rot90(r, axes = (1,2)) 
#                 label = np.rot90(label, axes = (1,2))
 
#             print(np.shape(ims),np.shape(labels))
            yield( [ims, labels])

#built network

input_im =keras.engine.Input( shape = [128,256,3], dtype = 'float32' )

l0 = keras.layers.convolutional.Conv2D( filters=64, kernel_size= (3,3),padding="same",     activation = 'relu' )(input_im)
l0 = keras.layers.convolutional.Conv2D( filters=64, kernel_size= (3,3),padding="same",     activation = 'relu' )(l0)

l1 = keras.layers.AvgPool2D(pool_size = (2,2))(l0)
l1 = keras.layers.convolutional.Conv2D( filters=128, kernel_size= (3,3),padding="same",     activation = 'relu' )(l1)
l1 = keras.layers.convolutional.Conv2D( filters=128, kernel_size= (3,3),padding="same",     activation = 'relu' )(l1)

l2 = keras.layers.AvgPool2D(pool_size = (2,2))(l1)
l2 = keras.layers.convolutional.Conv2D( filters=256, kernel_size= (3,3),padding="same",     activation = 'relu' )(l2)
l2 = keras.layers.convolutional.Conv2D( filters=256, kernel_size= (3,3),padding="same",     activation = 'relu' )(l2)

l3 = keras.layers.AvgPool2D(pool_size = (2,2))(l2)
l3 = keras.layers.convolutional.Conv2D( filters=512, kernel_size= (3,3),padding="same",     activation = 'relu' )(l3)
l3 = keras.layers.convolutional.Conv2D( filters=512, kernel_size= (3,3),padding="same",     activation = 'relu' )(l3)

l4 = keras.layers.AvgPool2D(pool_size = (2,2))(l3)
l4 = keras.layers.convolutional.Conv2D( filters=1024, kernel_size= (3,3),padding="same",     activation = 'relu' )(l4)
l4 = keras.layers.convolutional.Conv2D( filters=1024, kernel_size= (3,3),padding="same",     activation = 'relu' )(l4)


l3_up = keras.layers.convolutional.Conv2DTranspose(filters = 512 , kernel_size=(3,3) ,strides = (2, 2), padding="same")(l4)
l3_up = keras.layers.concatenate([l3,l3_up])
l3_up = keras.layers.convolutional.Conv2D( filters=512, kernel_size= (3,3),padding="same",     activation = 'relu' )(l3_up)
l3_up = keras.layers.convolutional.Conv2D( filters=512, kernel_size= (3,3),padding="same",     activation = 'relu' )(l3_up)

l2_up = keras.layers.convolutional.Conv2DTranspose(filters = 256 , kernel_size=(3,3) ,strides = (2, 2), padding="same")(l3_up)
l2_up = keras.layers.concatenate([l2,l2_up])
l2_up = keras.layers.convolutional.Conv2D( filters=256, kernel_size= (3,3),padding="same",     activation = 'relu' )(l2_up)
l2_up = keras.layers.convolutional.Conv2D( filters=256, kernel_size= (3,3),padding="same",     activation = 'relu' )(l2_up)

l1_up = keras.layers.convolutional.Conv2DTranspose(filters = 128 , kernel_size=(3,3) ,strides = (2, 2), padding="same")(l2_up)
l1_up = keras.layers.concatenate([l1,l1_up])
l1_up = keras.layers.convolutional.Conv2D( filters=128, kernel_size= (3,3),padding="same",     activation = 'relu' )(l1_up)
l1_up = keras.layers.convolutional.Conv2D( filters=128, kernel_size= (3,3),padding="same",     activation = 'relu' )(l1_up)

l0_up = keras.layers.convolutional.Conv2DTranspose(filters = 64 , kernel_size=(3,3) ,strides = (2, 2), padding="same")(l1_up)
l0_up = keras.layers.concatenate([l0,l0_up])
l0_up = keras.layers.convolutional.Conv2D( filters=64, kernel_size= (3,3),padding="same",     activation = 'relu' )(l0_up)
l0_up = keras.layers.convolutional.Conv2D( filters=64, kernel_size= (3,3),padding="same",     activation = 'relu' )(l0_up)

output = keras.layers.convolutional.Conv2D( filters=classes, kernel_size= (3,3),padding="same",     activation = 'sigmoid' )(l0_up)

model = keras.models.Model(inputs = input_im, outputs = output)

opt = keras.optimizers.adam( lr= 0.0001 , decay = 0,  clipnorm = 0.3 )
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics = ["accuracy"])

# #load a previous model
# model = keras.models.load_model( path +'model/model_rotations_3')


#train
for epoch in range(epochs):
    print(epoch)
    model.fit_generator(generator = generator(), steps_per_epoch = len(files_labels), epochs = 1 )

    if epoch % save_each == 0:
        name = path / (str(model_name)+str(epoch))
        model.save(name)


['Foto_315135_4.96788198177049_52.3608895136942.txt', 'Foto_324275_4.93026124514632_52.3607156232964.txt', 'Foto_363375_4.85767186254741_52.3384538670083.txt', 'Foto_388723_4.84628852935871_52.338713653074.txt', 'Foto_415954_4.85366247626555_52.4033462035686.txt', 'Foto_422880_4.84153984488568_52.3433816795177.txt', 'Foto_423299_4.84190916542411_52.345930217942.txt', 'Foto_565837_4.93938369794268_52.4077352083151.txt', 'Foto_584531_4.95616137031005_52.3985788534521.txt', 'Foto_595595_4.85295391520296_52.4035130139336.txt', 'Foto_597073_4.95549821102928_52.3438039090031.txt', 'Foto_597365_4.91149883260624_52.3286151030949.txt', 'Foto_598650_4.88982921940701_52.3366893727741.txt', 'Foto_599917_4.93976115995873_52.4072977814027.txt', 'Foto_600194_4.90589408406878_52.421546631874.txt', 'Foto_600495_4.86729094977073_52.4177009262639.txt', 'Foto_601824_4.89827879154395_52.3324519138533.txt']
0
Epoch 1/1


C:\Users\datalab1\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(1, 128, 256, 3)
(1, 128, 256, 3)
(1, 128, 256, 3)
(1, 128, 256, 3)
17/17 [==============================] - 121s 7s/step - loss: 0.9806 - acc: 0.7992
1
Epoch 1/1
(1, 128, 256, 3)
 1/17 [>.............................] - ETA: 2:03 - loss: 0.6254 - acc: 0.9630(1, 128, 256, 3)
(1, 128, 256, 3)
 3/17 [====>.........................] - ETA: 1:32 - loss: 0.3415 - acc: 0.9798(1, 128, 256, 3)
(1, 128, 256, 3)
 6/17 [=========>....................] - ETA: 1:11 - loss: 0.2538 - acc: 0.9848(1, 128, 256, 3)
(1, 128, 256, 3)
10/17 [================>.............] - ETA: 44s - loss: 0.2059 - acc: 0.9876(1, 128, 256, 3)
(1, 128, 256, 3)
14/17 [=======================>......] - ETA: 19s - loss: 0.2692 - acc: 0.9836(1, 128, 256, 3)
(1, 128, 256, 3)
15/17 [=========================>....] - ETA: 12s - loss: 0.2513 - acc: 0.9847(1, 128, 256, 3)
(1, 128, 256, 3)
17/17 [==============================] - 107s 6s/step - loss: 0.2586 - acc: 0.9842
2
Epoch 1/1
(1, 128, 256, 3)
 3/17 [====>.....................

In [50]:
print( path / (str(model_name)+str(1)))

\Users\datalab1\Lars\Panorama_Amsterdam\model\model_noreflect1


In [54]:
# from keras.utils import np_utils

# test=(np.arange(9)+1).reshape([3,3])

# print(test)
# test = np_utils.to_categorical(test, 10)
# print(test.shape)

import keras
from keras.utils import np_utils
import rasterio
import os,sys
import random
import numpy as np
import math
from pathlib import Path
import cv2
from skimage.transform import resize
import timeit

path = Path('/Users/datalab1/Lars/Panorama_Amsterdam/')
files_labels = os.listdir(os.path.join(path,'labeled' ))
label = os.path.join(path,'labeled',files_labels[2])
print('[label]',files_labels[2])
label = np.genfromtxt(label, delimiter = ',')
print(np.mean(label))
label = np.rint(resize(label, (128, 256))).astype(int)
print(np.mean(label))
print('label before one hot vector',np.shape(label))
#label = get_one_hot(label, classes)

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

[label] Foto_363375_4.85767186254741_52.3384538670083.txt
0.0370895
0.037811279296875
label before one hot vector (128, 256)


C:\Users\datalab1\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [56]:
imagepath = path / 'Target/'
print(os.listdir(imagepath))


['Foto_14303_4.84282538274236_52.3576999793185.jpg', 'Foto_14307_4.84310883017975_52.3577483832759.jpg', 'Foto_15211_4.84515022692867_52.3813948483496.jpg', 'Foto_305052_4.97221965926967_52.3674050455443.jpg', 'Foto_315135_4.96788198177049_52.3608895136942.jpg', 'Foto_315137_4.96800687250932_52.3608479128093.jpg', 'Foto_316801_4.97259407488883_52.3707821207791.jpg', 'Foto_324275_4.93026124514632_52.3607156232964.jpg', 'Foto_336001_4.97032824353945_52.36454177882.jpg', 'Foto_336053_4.96919578330943_52.3623104261626.jpg', 'Foto_336057_4.9691028351231_52.3621420090749.jpg', 'Foto_336263_4.9639195916302_52.3534638316559.jpg', 'Foto_336264_4.96387662262792_52.3534265392118.jpg', 'Foto_336720_4.95887501295395_52.347819728716.jpg', 'Foto_336836_4.95496163969845_52.3438842723432.jpg', 'Foto_339806_4.9475160808414_52.3388237762369.jpg', 'Foto_339809_4.94730943697132_52.3387744449981.jpg', 'Foto_343035_4.94163736011058_52.3364624868545.jpg', 'Foto_346093_4.89823339305518_52.3331253732173.jpg', '

In [20]:
%%timeit
test = np_utils.to_categorical(label, 5)

404 µs ± 44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
